In [3]:
import cv2
import sys
import time


In [4]:
class KCFTracker:
    """A wrapper for the KCF tracker."""
    
    def __init__(self):
        self.tracker = cv2.TrackerKCF_create()
        self.tracker_type = "KCF"

    def init(self, frame, bbox):
        """Initialize the tracker."""
        return self.tracker.init(frame, bbox)

    def update(self, frame):
        """
        Update the tracker with a new frame.
        Returns: (success, bbox, fps)
        """
        start_time = time.time()
        success, bbox = self.tracker.update(frame)
        end_time = time.time()
        
        # Calculate FPS for this single update step
        fps = 1.0 / (end_time - start_time) if (end_time - start_time) > 0 else 0
        return success, bbox, fps

In [5]:
class CSRTTracker:
    """A wrapper for the CSRT tracker."""

    def __init__(self):
        self.tracker = cv2.TrackerCSRT_create()
        self.tracker_type = "CSRT"

    def init(self, frame, bbox):
        """Initialize the tracker."""
        return self.tracker.init(frame, bbox)

    def update(self, frame):
        """
        Update the tracker with a new frame.
        Returns: (success, bbox, fps)
        """
        start_time = time.time()
        success, bbox = self.tracker.update(frame)
        end_time = time.time()
        
        # Calculate FPS for this single update step
        fps = 1.0 / (end_time - start_time) if (end_time - start_time) > 0 else 0
        return success, bbox, fps

In [30]:
VIDEO_SOURCE = "f1_cut.mp4"  # !!! Change this to your video file or 0 for webcam
OUTPUT_FILE = "csrt.mp4"
DISPLAY_WIDTH = 720

In [31]:
cap = cv2.VideoCapture(VIDEO_SOURCE)
ret, frame = cap.read()

frame_height, frame_width = frame.shape[:2]
aspect_ratio = frame_height / frame_width
display_height = int(DISPLAY_WIDTH * aspect_ratio)
display_dim = (DISPLAY_WIDTH, display_height)
scale_x = frame_width / DISPLAY_WIDTH
scale_y = frame_height / display_height
frame_small = cv2.resize(frame, display_dim, interpolation=cv2.INTER_AREA)

bbox = cv2.selectROI("Select Object to Track", frame_small, False)

x_sm, y_sm, w_sm, h_sm = bbox
bbox = (
    int(x_sm * scale_x),
    int(y_sm * scale_y),
    int(w_sm * scale_x),
    int(h_sm * scale_y)
)

cv2.destroyWindow("Select Object to Track")
tracker_csrt = CSRTTracker()
tracker_csrt.init(frame, bbox)
frame_height, frame_width = frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_FILE, fourcc, 30.0, (frame_width, frame_height))
kcf_times = []
csrt_times = []
while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break  # End of video



    draw_frame = frame.copy()
    ok_csrt, bbox_csrt, fps_csrt = tracker_csrt.update(frame)
    csrt_times.append(fps_csrt)
    p1 = (int(bbox_csrt[0]), int(bbox_csrt[1]))
    p2 = (int(bbox_csrt[0] + bbox_csrt[2]), int(bbox_csrt[1] + bbox_csrt[3]))
    cv2.rectangle(draw_frame, p1, p2, (0, 0, 255), 2, 1)
    cv2.putText(draw_frame, "CSRT", (p1[0], p1[1] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(draw_frame, f"CSRT FPS: {int(fps_csrt)}", (20, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    display_frame = cv2.resize(draw_frame, display_dim, interpolation=cv2.INTER_AREA)

    cv2.imshow("Tracking Comparison", display_frame)
    out.write(draw_frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [28]:
VIDEO_SOURCE = "f1_cut.mp4"  # !!! Change this to your video file or 0 for webcam
OUTPUT_FILE = "kcf.mp4"
DISPLAY_WIDTH = 720

In [29]:
cap = cv2.VideoCapture(VIDEO_SOURCE)
ret, frame = cap.read()

frame_height, frame_width = frame.shape[:2]
aspect_ratio = frame_height / frame_width
display_height = int(DISPLAY_WIDTH * aspect_ratio)
display_dim = (DISPLAY_WIDTH, display_height)
scale_x = frame_width / DISPLAY_WIDTH
scale_y = frame_height / display_height
frame_small = cv2.resize(frame, display_dim, interpolation=cv2.INTER_AREA)

bbox = cv2.selectROI("Select Object to Track", frame_small, False)

x_sm, y_sm, w_sm, h_sm = bbox
bbox = (
    int(x_sm * scale_x),
    int(y_sm * scale_y),
    int(w_sm * scale_x),
    int(h_sm * scale_y)
)

cv2.destroyWindow("Select Object to Track")
tracker_csrt = KCFTracker()
tracker_csrt.init(frame, bbox)
frame_height, frame_width = frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_FILE, fourcc, 30.0, (frame_width, frame_height))
kcf_times = []
csrt_times = []
while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break  # End of video



    draw_frame = frame.copy()
    ok_csrt, bbox_csrt, fps_csrt = tracker_csrt.update(frame)
    csrt_times.append(fps_csrt)
    p1 = (int(bbox_csrt[0]), int(bbox_csrt[1]))
    p2 = (int(bbox_csrt[0] + bbox_csrt[2]), int(bbox_csrt[1] + bbox_csrt[3]))
    cv2.rectangle(draw_frame, p1, p2, (0, 0, 255), 2, 1)
    cv2.putText(draw_frame, "KCF", (p1[0], p1[1] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(draw_frame, f"KCF FPS: {int(fps_csrt)}", (20, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    display_frame = cv2.resize(draw_frame, display_dim, interpolation=cv2.INTER_AREA)

    cv2.imshow("Tracking Comparison", display_frame)
    out.write(draw_frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

## Tracking stability
#### - CRST is more accurate and stablity, KCF is simple algorithm
## Frame Rate
#### - CRST is get low FPS because it requried expensive resource at start vidios it around 50 FPS, KCF got high FPS around 500++ FPS
## Failure Case
#### - Both CRST and KCF get loss tracking object but CRST loss object after camera move while KCF loss tracking when object move fast 
 